In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import os, sys
import torch
from imageio import imread
from matplotlib import pyplot as plt
import numpy as np
from models.EdgeModel import EdgeCompleter, ConfigManager
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 2 to use
sys.path.append("/home/files")

In [3]:
#data load
from datasets.StanfordOnlineProducts import load_flist
from datasets.irregular_masks import load_mask_flist
from data.EdgeData import MaskedEdgeDataset

In [4]:
X, Y = load_flist(split='train')
masks = load_mask_flist(split='train')

In [5]:
# train_x, train_y = X[:5000], Y[:5000]
# val_x, val_y = X[5000:5500], Y[5000:5500]
train_x, train_y = X[:50000], Y[:50000]
val_x, val_y = X[50000:], Y[50000:]
train_ds = MaskedEdgeDataset(train_x, irregular_mask_flist=masks).create_iterator()
val_ds = MaskedEdgeDataset(val_x, irregular_mask_flist=masks).create_iterator()

In [6]:
config = ConfigManager()
config.GPU = [0]
config.GAN_LOSS = "hinge"
edge_completer = EdgeCompleter(config)

In [7]:
from utils.train import EarlyStopping, TensorBoard
prj_path = "/home/files/lab/experiments/EdgeCompleter/exp2_only_irregular_mask"
tensorboard = TensorBoard(path=prj_path)
edge_completer.exp_path = prj_path

In [ ]:
for epoch in range(1,1000):
    metrics = {}
    metrics["train"] = edge_completer.train_steps(train_ds)
    metrics['val'] = edge_completer.test_steps(val_ds, epoch)
    torch.save({"generator" : edge_completer.generator.state_dict(),
                "discriminator" : edge_completer.discriminator.state_dict()},
                f"{prj_path}/checkpoint-{epoch}.pth")
    tensorboard(metrics, epoch)

10 steps 0.633 secs => gen loss : 0.759, dis_loss : 1.000
20 steps 0.638 secs => gen loss : 0.415, dis_loss : 0.999
30 steps 0.635 secs => gen loss : 0.256, dis_loss : 1.000
40 steps 0.635 secs => gen loss : 0.147, dis_loss : 0.999
50 steps 0.637 secs => gen loss : -0.078, dis_loss : 0.999


# Load

In [ ]:
state_dict = torch.load(f"{prj_path}/checkpoint-100.pth")

In [ ]:
edge_completer.generator.load_state_dict(state_dict["generator"])
edge_completer.discriminator.load_state_dict(state_dict["discriminator"])

In [ ]:
items = next(iter(val_ds))

In [ ]:
images, grays, edges, masks = items

In [ ]:
images, grays, edges, masks = (item.to(edge_completer.cuda) for item in items)

In [ ]:
results = edge_completer.forward(grays, edges, masks)

In [ ]:
for image, gray, edge, mask, result in zip(images, grays, edges, masks, results):
    plt.subplot(1,3,1)
    edges_masked = (edge * (1 - mask))
    plt.imshow(edges_masked.to("cpu")[0], cmap='gray')
    plt.subplot(1,3,2)
    plt.imshow(edge.to("cpu")[0], cmap='gray')
    plt.subplot(1,3,3)
    plt.imshow(result.detach().to("cpu")[0], cmap='gray')
    plt.show()

In [ ]:
results[0].shape

In [ ]:
plt.imshow(edges_masked.to("cpu")[0])